In [1]:
import numpy as np
import xgboost as xgb
from wideboost.wrappers import wxgb
from wideboost.wrappers.wxgb import wxgbModel
from matplotlib import pyplot as plt

from sklearn import datasets

In [2]:
digits = datasets.load_digits()

idx_train = np.random.choice(np.arange(1797),round(1797*0.7),replace=False)
idx_test = np.setdiff1d(np.arange(1797),idx_train)


xtrain = digits['data']
ytrain = digits['target']

dtrain = xgb.DMatrix(xtrain[idx_train,:],label=ytrain[idx_train])
dtest = xgb.DMatrix(xtrain[idx_test,:],label=ytrain[idx_test])

In [11]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    wbst = wxgb.train(param, dtrain, 100,watchlist,evals_result=ed1_results)
    
    output = min(ed1_results['test']['merror'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['R','I','Rn','In']),
    'extra_dims': hp.choice('extra_dims',range(16)),
    'objective': hp.choice('objective',['multi:softmax']),
    'eval_metric':hp.choice('eval_metric',[['merror']]),
    'num_class':hp.choice('num_class',[10]),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.choice('max_depth',range(1,11)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'alpha': 0, 'btype': 'Rn', 'colsample_bylevel': 0.994491324055232, 'colsample_bytree': 0.8247844351304049, 'eta': 0.3363438110461667, 'eval_metric': ('merror',), 'extra_dims': 10, 'gamma': 0, 'lambda': 0, 'max_depth': 6, 'min_child_weight': 1.3631100912661176e-06, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.9572607156686302}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `obj` in train.
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                
Setting param `disable_default_eval_metric` to 1.      
[09:55:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you f

[4]	train-merror:0.00954	test-merror:0.04453                           

[5]	train-merror:0.00556	test-merror:0.04453                           

[6]	train-merror:0.00318	test-merror:0.03896                           

[7]	train-merror:0.00318	test-merror:0.03154                           

[8]	train-merror:0.00238	test-merror:0.02968                           

[9]	train-merror:0.00080	test-merror:0.03154                           

[10]	train-merror:0.00159	test-merror:0.03154                          

[11]	train-merror:0.00080	test-merror:0.04082                          

[12]	train-merror:0.00000	test-merror:0.03711                          

[13]	train-merror:0.00000	test-merror:0.03896                          

[14]	train-merror:0.00000	test-merror:0.03525                          

[15]	train-merror:0.00000	test-merror:0.03525                          

[16]	train-merror:0.00000	test-merror:0.03525                          

[17]	train-merror:0.00000	test-merror:0.03711      

[0]	train-merror:0.02146	test-merror:0.11132                           

[1]	train-merror:0.00238	test-merror:0.05380                           

[2]	train-merror:0.00080	test-merror:0.04453                           

[3]	train-merror:0.00080	test-merror:0.04082                           

[4]	train-merror:0.00080	test-merror:0.04082                           

[5]	train-merror:0.00000	test-merror:0.03896                           

[6]	train-merror:0.00000	test-merror:0.03711                           

[7]	train-merror:0.00000	test-merror:0.03896                           

[8]	train-merror:0.00000	test-merror:0.03711                           

[9]	train-merror:0.00000	test-merror:0.03525                           

[10]	train-merror:0.00000	test-merror:0.03340                          

[11]	train-merror:0.00000	test-merror:0.02968                          

[12]	train-merror:0.00000	test-merror:0.02968                          

[13]	train-merror:0.00000	test-merror:0.02783      

[0]	train-merror:0.01431	test-merror:0.06494                           

[1]	train-merror:0.00556	test-merror:0.05380                           

[2]	train-merror:0.00238	test-merror:0.04453                           

[3]	train-merror:0.00159	test-merror:0.04267                           

[4]	train-merror:0.00080	test-merror:0.04267                           

[5]	train-merror:0.00080	test-merror:0.04638                           

[6]	train-merror:0.00080	test-merror:0.04267                           

[7]	train-merror:0.00080	test-merror:0.04267                           

[8]	train-merror:0.00080	test-merror:0.04453                           

[9]	train-merror:0.00080	test-merror:0.04082                           

[10]	train-merror:0.00080	test-merror:0.04082                          

[11]	train-merror:0.00080	test-merror:0.04082                          

[12]	train-merror:0.00080	test-merror:0.04082                          

[13]	train-merror:0.00080	test-merror:0.03896      

[0]	train-merror:0.54134	test-merror:0.55288                           

[1]	train-merror:0.46582	test-merror:0.45826                           

[2]	train-merror:0.43164	test-merror:0.44341                           

[3]	train-merror:0.40541	test-merror:0.42672                           

[4]	train-merror:0.39110	test-merror:0.41187                           

[5]	train-merror:0.39269	test-merror:0.40074                           

[6]	train-merror:0.36884	test-merror:0.38590                           

[7]	train-merror:0.37838	test-merror:0.38776                           

[8]	train-merror:0.37758	test-merror:0.38776                           

[9]	train-merror:0.35533	test-merror:0.37291                           

[10]	train-merror:0.34897	test-merror:0.36364                          

[11]	train-merror:0.34579	test-merror:0.34508                          

[12]	train-merror:0.34420	test-merror:0.35065                          

[13]	train-merror:0.34261	test-merror:0.35065      

...

[26]	train-merror:0.00000	test-merror:0.01670                           

[27]	train-merror:0.00000	test-merror:0.01670                           

[28]	train-merror:0.00000	test-merror:0.01670                           

[29]	train-merror:0.00000	test-merror:0.01484                           

[30]	train-merror:0.00000	test-merror:0.01299                           

[31]	train-merror:0.00000	test-merror:0.01299                           

[32]	train-merror:0.00000	test-merror:0.01299                           

[33]	train-merror:0.00000	test-merror:0.01299                           

[34]	train-merror:0.00000	test-merror:0.01299                           

[35]	train-merror:0.00000	test-merror:0.01113                           

[36]	train-merror:0.00000	test-merror:0.01113                           

[37]	train-merror:0.00000	test-merror:0.01113                           

[38]	train-merror:0.00000	test-merror:0.01299                           

[39]	train-merror:0.00000	test-merror:

[22]	train-merror:0.01192	test-merror:0.04453                           

[23]	train-merror:0.01272	test-merror:0.04638                           

[24]	train-merror:0.01113	test-merror:0.04453                           

[25]	train-merror:0.01192	test-merror:0.04267                           

[26]	train-merror:0.01272	test-merror:0.04267                           

[27]	train-merror:0.01272	test-merror:0.04267                           

[28]	train-merror:0.01192	test-merror:0.04267                           

[29]	train-merror:0.01192	test-merror:0.04267                           

[30]	train-merror:0.01192	test-merror:0.04267                           

[31]	train-merror:0.01033	test-merror:0.04082                           

[32]	train-merror:0.01033	test-merror:0.04267                           

[33]	train-merror:0.01033	test-merror:0.04267                           

[34]	train-merror:0.01192	test-merror:0.04267                           

[35]	train-merror:0.01113	test-merror:

[17]	train-merror:0.00000	test-merror:0.02226                           

[18]	train-merror:0.00000	test-merror:0.02226                           

[19]	train-merror:0.00000	test-merror:0.02226                           

[20]	train-merror:0.00000	test-merror:0.02226                           

[21]	train-merror:0.00000	test-merror:0.02412                           

[22]	train-merror:0.00000	test-merror:0.02412                           

[23]	train-merror:0.00000	test-merror:0.02412                           

[24]	train-merror:0.00000	test-merror:0.02412                           

[25]	train-merror:0.00000	test-merror:0.02226                           

[26]	train-merror:0.00000	test-merror:0.02226                           

[27]	train-merror:0.00000	test-merror:0.02041                           

[28]	train-merror:0.00000	test-merror:0.02041                           

[29]	train-merror:0.00000	test-merror:0.02041                           

[30]	train-merror:0.00000	test-merror:

[13]	train-merror:0.00000	test-merror:0.02597                           

[14]	train-merror:0.00000	test-merror:0.02783                           

[15]	train-merror:0.00000	test-merror:0.02226                           

[16]	train-merror:0.00000	test-merror:0.02226                           

[17]	train-merror:0.00000	test-merror:0.02412                           

[18]	train-merror:0.00000	test-merror:0.02226                           

[19]	train-merror:0.00000	test-merror:0.01855                           

[20]	train-merror:0.00000	test-merror:0.01855                           

[21]	train-merror:0.00000	test-merror:0.02041                           

[22]	train-merror:0.00000	test-merror:0.01670                           

[23]	train-merror:0.00000	test-merror:0.01484                           

[24]	train-merror:0.00000	test-merror:0.01670                           

[25]	train-merror:0.00000	test-merror:0.01484                           

[26]	train-merror:0.00000	test-merror:

[9]	train-merror:0.06359	test-merror:0.09091                            

[10]	train-merror:0.05723	test-merror:0.09276                           

[11]	train-merror:0.05326	test-merror:0.08905                           

[12]	train-merror:0.05246	test-merror:0.08534                           

[13]	train-merror:0.04690	test-merror:0.08905                           

[14]	train-merror:0.04451	test-merror:0.07978                           

[15]	train-merror:0.03975	test-merror:0.08163                           

[16]	train-merror:0.03736	test-merror:0.07607                           

[17]	train-merror:0.03816	test-merror:0.07978                           

[18]	train-merror:0.03418	test-merror:0.07792                           

[19]	train-merror:0.03339	test-merror:0.07236                           

[20]	train-merror:0.03021	test-merror:0.07050                           

[21]	train-merror:0.02703	test-merror:0.06865                           

[22]	train-merror:0.02464	test-merror:

In [12]:
print(best_val)
print(space_eval(spc, best))

0.007421
{'alpha': 0, 'btype': 'In', 'colsample_bylevel': 0.8312798398436857, 'colsample_bytree': 0.760621527198531, 'eta': 0.06320324278575613, 'eval_metric': ('merror',), 'extra_dims': 11, 'gamma': 0, 'lambda': 0, 'max_depth': 3, 'min_child_weight': 0.0001393071794052514, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.5333605322257972}
